In [0]:
print('Taking latest updated folder name from Azure blob storage')
dbutils.widgets.text("folderName", "","")
folderName = dbutils.widgets.get("folderName")

Taking latest updated folder name from Azure blob storage


In [0]:
import re
from datetime import datetime, timedelta
from pyspark.sql.functions import current_date, col, when, lit, current_timestamp

In [0]:
print('Connecting to azure blob storage')

Connecting to azure blob storage


In [0]:
spark.conf.set(
    "fs.azure.account.key.azdestabm001.dfs.core.windows.net",
    dbutils.secrets.get(scope="az-blob-storage-account_key", key="az-de-st-key-01"))

In [0]:
list_of_Files =  dbutils.fs.ls(f"abfss://hotel-booking-raw-data@azdestabm001.dfs.core.windows.net/{folderName}")
Schema = 'hotel_booking_mart'

In [0]:
for file in list_of_Files:

    # getting all variables
    file_path = file.path
    file_name = file.name
    table_name = ''.join([x.replace("_", "").title() for x in re.findall('[a-z]*_', file_name.lower())])
    
    # reading df
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df = df.withColumn('ingestion_time', current_timestamp())
    df = df.withColumn('file_path', lit(file_path))    
    
    try:
        table_df = spark.sql(f'select * from {Schema}.{table_name}')
        if table_df.count() > 0:
            # doing full refresh
            df.write.mode('overwrite').saveAsTable(f'{Schema}.{table_name}')
    except Exception:        
        # means table does not exists    
        df.write.mode('overwrite').saveAsTable(f'{Schema}.{table_name}')